In [45]:
import numpy as np
import pandas as pd

d0 = "king palace jungle sleeps"
d1 = "jungle lion fire king discovery"
d2 = "king timon musafa"
d3 = "timon lion simba"

q = "lion king"

D={d0, d1, d2, d3}

In [46]:
def my_TD_matrix(d1, d2, d3, d4, q):
    combined_lists = d1.split() + d2.split() + d3.split() + d4.split() + q.split()
    terms = sorted(set(combined_lists))
    term_document_matrix = {}
    for term in terms:
        term_matrix = []
        document1_term_count = [1 if term in d1 else 0]
        document2_term_count = [1 if term in d2 else 0]
        document3_term_count = [1 if term in d3 else 0]
        document4_term_count = [1 if term in d4 else 0]
        query_term_count = [1 if term in q else 0]
        
        term_matrix = document1_term_count + document2_term_count + document3_term_count + document4_term_count + query_term_count
        term_document_matrix[term] = term_matrix
    
    term_document_matrix = pd.DataFrame(term_document_matrix).T
    term_document_matrix = term_document_matrix.rename(columns={0: 'd0', 1: 'd1', 2: 'd2', 3: 'd3', 4: 'query'})

    return term_document_matrix

In [47]:
term_document_matrix = my_TD_matrix(d0, d1, d2, d3, q)
term_document_matrix

,d0,d1,d2,d3,query
discovery,0,1,0,0,0
fire,0,1,0,0,0
jungle,1,1,0,0,0
king,1,1,1,0,1
lion,0,1,0,1,1
musafa,0,0,1,0,0
palace,1,0,0,0,0
simba,0,0,0,1,0
sleeps,1,0,0,0,0
timon,0,0,1,1,0


In [48]:
#Document Frequency
term_document_matrix['docfreq'] = term_document_matrix['d0'] + term_document_matrix['d1'] + term_document_matrix['d2'] + term_document_matrix['d3']
term_document_matrix

,d0,d1,d2,d3,query,docfreq
discovery,0,1,0,0,0,1
fire,0,1,0,0,0,1
jungle,1,1,0,0,0,2
king,1,1,1,0,1,3
lion,0,1,0,1,1,2
musafa,0,0,1,0,0,1
palace,1,0,0,0,0,1
simba,0,0,0,1,0,1
sleeps,1,0,0,0,0,1
timon,0,0,1,1,0,2


In [49]:
term_document_matrix['global_weights'] = np.log10((len(D))/term_document_matrix['docfreq'])

In [50]:
#document and query vector
term_document_matrix['d0_tfidf'] = term_document_matrix['global_weights']*term_document_matrix['d0']
term_document_matrix['d1_tfidf'] = term_document_matrix['global_weights']*term_document_matrix['d1']
term_document_matrix['d2_tfidf'] = term_document_matrix['global_weights']*term_document_matrix['d2']
term_document_matrix['d3_tfidf'] = term_document_matrix['global_weights']*term_document_matrix['d3']
term_document_matrix['query_tfidf'] = term_document_matrix['global_weights']*term_document_matrix['query']

In [51]:
term_document_matrix

,d0,d1,d2,d3,query,docfreq,global_weights,d0_tfidf,d1_tfidf,d2_tfidf,d3_tfidf,query_tfidf
discovery,0,1,0,0,0,1,0.602060,0.000000,0.602060,0.000000,0.00000,0.000000
fire,0,1,0,0,0,1,0.602060,0.000000,0.602060,0.000000,0.00000,0.000000
jungle,1,1,0,0,0,2,0.301030,0.301030,0.301030,0.000000,0.00000,0.000000
king,1,1,1,0,1,3,0.124939,0.124939,0.124939,0.124939,0.00000,0.124939
lion,0,1,0,1,1,2,0.301030,0.000000,0.301030,0.000000,0.30103,0.301030
musafa,0,0,1,0,0,1,0.602060,0.000000,0.000000,0.602060,0.00000,0.000000
palace,1,0,0,0,0,1,0.602060,0.602060,0.000000,0.000000,0.00000,0.000000
simba,0,0,0,1,0,1,0.602060,0.000000,0.000000,0.000000,0.60206,0.000000
sleeps,1,0,0,0,0,1,0.602060,0.602060,0.000000,0.000000,0.00000,0.000000
timon,0,0,1,1,0,2,0.301030,0.000000,0.000000,0.301030,0.30103,0.000000


In [53]:
#Rocchio feedback
def rocchio(df,alpha,beta,gamma):
    q_updates = (alpha*df['query_tfidf']) + (beta*df['d1_tfidf']) - (gamma*df['d3_tfidf'])
    return q_updates

In [54]:
rocchio(term_document_matrix,1,0.8,0.1)

discovery    0.481648
fire         0.481648
jungle       0.240824
king         0.224890
lion         0.511751
musafa       0.000000
palace       0.000000
simba       -0.060206
sleeps       0.000000
timon       -0.030103
dtype: float64

In [55]:
rocchio(term_document_matrix,1,0.1,0.9)

discovery    0.060206
fire         0.060206
jungle       0.030103
king         0.137433
lion         0.060206
musafa       0.000000
palace       0.000000
simba       -0.541854
sleeps       0.000000
timon       -0.270927
dtype: float64

In [56]:
rocchio(term_document_matrix,1,1,1)

discovery    0.602060
fire         0.602060
jungle       0.301030
king         0.249877
lion         0.301030
musafa       0.000000
palace       0.000000
simba       -0.602060
sleeps       0.000000
timon       -0.301030
dtype: float64